In [148]:
import numpy as np
import pandas as pd
import scipy.sparse as sps

In [144]:
data = pd.io.parsers.read_csv('../../data/MovieLens_1M/ratings.dat', 
    names=['user_id', 'movie_id', 'rating', 'time'],
    engine='python', delimiter='::')
movie_data = pd.io.parsers.read_csv('../../data/MovieLens_1M/movies.dat',
    names=['movie_id', 'title', 'genre'],
    engine='python', delimiter='::')

In [146]:
data.rating.mean()

3.581564453029317

In [181]:
data

,user_id,movie_id,rating,time
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [200]:
sparse = sps.csr_matrix((data.rating, (data.user_id-1, data.movie_id-1)))

In [201]:
ratings_mat = sparse.toarray()

In [203]:
ratings_mat[6039][1096]

4

In [213]:
normalised_mat = ratings_mat - np.asarray([(np.mean(ratings_mat, 1))]).T

In [214]:
# A = normalised_mat.T / np.sqrt(ratings_mat.shape[0])
U, S, V = np.linalg.svd(normalised_mat)

In [215]:
# print(A.shape)
print(U.shape)
print(S.shape)
print(V.shape)

(6040, 6040)
(3952,)
(3952, 3952)


In [216]:
def top_cosine_similarity(data, movie_id, top_n=10):
    index = movie_id # Movie id starts from 1
    movie_row = data[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data))
    similarity = np.dot(movie_row, data.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity)
    return sort_indexes[:top_n]

# Helper function to print top N similar movies
def print_similar_movies(movie_data, movie_id, top_indexes):
    print('Recommendations for {0}: \n'.format(
    movie_data[movie_data.movie_id == movie_id].title.values[0]))
    for id in top_indexes + 1:
        print(movie_data[movie_data.movie_id == id].title.values[0])

In [217]:
k = 50
movie_id = 1 # Grab an id from movies.dat
top_n = 10

sliced = V.T[:, :k] # representative data
indexes = top_cosine_similarity(sliced, movie_id, top_n)
print_similar_movies(movie_data, movie_id, indexes)

Recommendations for Toy Story (1995): 

Jumanji (1995)
Hook (1991)
Indian in the Cupboard, The (1995)
NeverEnding Story II: The Next Chapter, The (1990)
Dragonheart (1996)
NeverEnding Story, The (1984)
Santa Clause, The (1994)
Labyrinth (1986)
Borrowers, The (1997)
Flubber (1997)


In [218]:
normalised_mat = ratings_mat - np.matrix(np.mean(ratings_mat, 1)).T
cov_mat = np.cov(normalised_mat)
evals, evecs = np.linalg.eig(cov_mat)

In [219]:
k = 50
movie_id = 1 # Grab an id from movies.dat
top_n = 10

sliced = evecs[:, :k] # representative data
top_indexes = top_cosine_similarity(sliced, movie_id, top_n)
print_similar_movies(movie_data, movie_id, top_indexes)

Recommendations for Toy Story (1995): 

Jumanji (1995)


IndexError: index 0 is out of bounds for axis 0 with size 0